# A Gentle Introduction to RAG Applications

This notebook creates a simple RAG (Retrieval-Augmented Generation) system to answer questions from a PDF document using an open-source model.

In [2]:
PDF_FILE = "paul.pdf"

# We'll be using Llama 3.1 8B for this example.
# MODEL = "llama3.1"

# We'll be using Gemma 2B for this example.
MODEL = "gemma:2b"

## Loading the PDF document

Let's start by loading the PDF document and breaking it down into separate pages.

<img src='images/documents.png' width="1000">

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(PDF_FILE)
pages = loader.load()

print(f"Number of pages: {len(pages)}")
print(f"Length of a page: {len(pages[1].page_content)}")
print("Content of a page:", pages[1].page_content[:50])

Number of pages: 9
Length of a page: 3272
Content of a page: 10% a week. And while 110 may not seem much better


## Splitting the pages in chunks

Pages are too long, so let's split pages into different chunks.

<img src='images/splitter.png' width="1000">


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)

chunks = splitter.split_documents(pages)
print(f"Number of chunks: {len(chunks)}")
print(f"Length of a chunk: {len(chunks[1].page_content)}")
print("Content of a chunk:", chunks[1].page_content)


Number of chunks: 23
Length of a chunk: 1236
Content of a chunk: took better advantage of it than Stripe. At YC we use the term
"Collison installation" for the technique they invented. More
diffident founders ask "Will you try our beta?" and if the answer is
yes, they say "Great, we'll send you a link." But the Collison
brothers weren't going to wait. When anyone agreed to try Stripe
they'd say "Right then, give me your laptop" and set them up on
the spot.
There are two reasons founders resist going out and recruiting
users individually. One is a combination of shyness and laziness.
They'd rather sit at home writing code than go out and talk to a
bunch of strangers and probably be rejected by most of them.
But for a startup to succeed, at least one founder (usually the
CEO) will have to spend a lot of time on sales and marketing. [2]
The other reason founders ignore this path is that the absolute
numbers seem so small at first. This can't be how the big, famous
startups got started, th

## Storing the chunks in a vector store

We can now generate embeddings for every chunk and store them in a vector store.

<img src='images/vectorstore.png' width="1000">


In [6]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model=MODEL)
vectorstore = FAISS.from_documents(chunks, embeddings)

## Setting up a retriever

We can use a retriever to find chunks in the vector store that are similar to a supplied question.

<img src='images/retriever.png' width="1000">



In [7]:
retriever = vectorstore.as_retriever()
retriever.invoke("What can you get away with when you only have a small number of users?")

[Document(id='afbe345d-ad11-42f2-864b-6342602ca448', metadata={'source': 'paul.pdf', 'page': 3}, page_content='Experience\nI was trying to think of a phrase to convey how extreme your\nattention to users should be, and I realized Steve Jobs had\nalready done it: insanely great. Steve wasn\'t just using "insanely"\nas a synonym for "very." He meant it more literally — that one\nshould focus on quality of execution to a degree that in everyday\nlife would be considered pathological.\nAll the most successful startups we\'ve funded have, and that\nprobably doesn\'t surprise would-be founders. What novice\nfounders don\'t get is what insanely great translates to in a larval\nstartup. When Steve Jobs started using that phrase, Apple was\nalready an established company. He meant the Mac (and its\ndocumentation and even packaging — such is the nature of\nobsession) should be insanely well designed and manufactured.\nThat\'s not hard for engineers to grasp. It\'s just a more extreme\nversion of

## Configuring the model

We'll be using Ollama to load the local model in memory. After creating the model, we can invoke it with a question to get the response back.

<img src='images/model.png' width="1000">

In [8]:
from langchain_ollama import ChatOllama

model = ChatOllama(model=MODEL, temperature=0)
model.invoke("Who is the president of the United States?")

AIMessage(content='I do not have access to real-time information and cannot provide the current president of the United States. For the most up-to-date information, please check a reputable news source or government website.', additional_kwargs={}, response_metadata={'model': 'gemma:2b', 'created_at': '2024-12-26T06:16:44.33918308Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3546985669, 'load_duration': 23763432, 'prompt_eval_count': 31, 'prompt_eval_duration': 641000000, 'eval_count': 42, 'eval_duration': 2880000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-44b31388-a7f9-4f95-aa12-e3deb78f4b8a-0', usage_metadata={'input_tokens': 31, 'output_tokens': 42, 'total_tokens': 73})

## Parsing the model's response

The response from the model is an `AIMessage` instance containing the answer. We can extract the text answer by using the appropriate output parser. We can connect the model and the parser using a chain.

<img src='images/parser.png' width="1000">


In [9]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
print(chain.invoke("Who is the president of the United States?"))

I do not have access to real-time information and cannot provide the current president of the United States. For the most up-to-date information, please check a reputable news source or government website.


## Setting up a prompt

In addition to the question we want to ask, we also want to provide the model with the context from the PDF file. We can use a prompt template to define and reuse the prompt we'll use with the model.


<img src='images/prompt.png' width="1000">

In [10]:
from langchain.prompts import PromptTemplate

template = """
You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: Here is some context

Question: Here is a question



## Adding the prompt to the chain

We can now chain the prompt with the model and the parser.

<img src='images/chain1.png' width="1000">

In [11]:
chain = prompt | model | parser

chain.invoke({
    "context": "Anna's sister is Susan", 
    "question": "Who is Susan's sister?"
})


"Anna's sister is Susan.\n\n"

## Adding the retriever to the chain

Finally, we can connect the retriever to the chain to get the context from the vector store.

<img src='images/chain2.png' width="1000">

In [12]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

## Using the chain to answer questions

Finally, we can use the chain to ask questions that will be answered using the PDF document.

In [13]:
questions = [
    "What can you get away with when you only have a small number of users?",
    "What's the most common unscalable thing founders have to do at the start?",
    "What's one of the biggest things inexperienced founders and investors get wrong about startups?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print("*************************\n")

Question: What can you get away with when you only have a small number of users?
Answer: According to the context, you can fabricate things yourself to turn your product into a hit. This is especially helpful for hardware startups who have to deal with minimum order requirements from investors.
*************************

Question: What's the most common unscalable thing founders have to do at the start?
Answer: According to the context, the most common unscalable thing founders have to do at the start is assembling their product themselves.
*************************

Question: What's one of the biggest things inexperienced founders and investors get wrong about startups?
Answer: One of the biggest things inexperienced founders and investors get wrong about startups is that they're not as important as they think they are. They often dismiss startups as a necessary evil, but they can actually have a significant impact on the world.
*************************

